In [60]:
# predict which theme ...
import pandas as pd
import numpy as np

In [61]:
# read data -- sets
setsfile = '~/Documents/MSofDS/Spring2018/DS-Project2/lego-database/sets.csv'
sets = pd.read_csv(setsfile)
#sets.head()

In [62]:
themefile = '~/Documents/MSofDS/Spring2018/DS-Project2/lego-database/themes.csv'
themes = pd.read_csv(themefile)
#themes.head()

In [63]:
setsinvenfile = '~/Documents/MSofDS/Spring2018/DS-Project2/lego-database/inventory_sets.csv'
invenotry_sets = pd.read_csv(setsinvenfile)

In [64]:
invenfile = '~/Documents/MSofDS/Spring2018/DS-Project2/lego-database/inventories.csv'
invenotries = pd.read_csv(invenfile)
#invenotries.head()

In [65]:
invenpartsfile = '~/Documents/MSofDS/Spring2018/DS-Project2/lego-database/inventory_parts.csv'
invenotry_parts = pd.read_csv(invenpartsfile)

In [66]:
partsfile = '~/Documents/MSofDS/Spring2018/DS-Project2/lego-database/parts.csv'
parts = pd.read_csv(partsfile)
#parts.head()

In [67]:
colorsfile = '~/Documents/MSofDS/Spring2018/DS-Project2/lego-database/colors.csv'
colors = pd.read_csv(colorsfile)

In [68]:
partcatfile = '~/Documents/MSofDS/Spring2018/DS-Project2/lego-database/part_categories.csv'
part_category = pd.read_csv(partcatfile)
#part_category.head()

In [69]:
# given the information about parts, predict the theme of a set

In [70]:
#join sets and themes together
t1 = pd.merge(sets,themes, left_on = 'theme_id', right_on = 'id', suffixes = ('_set','_theme'))

In [71]:
# join t1 with ivnetories, inventoryset inventory parts
t4 = pd.merge(t1,invenotry_sets, on = 'set_num')
#t4.head()

In [72]:
t4 = t4.drop(t4.columns[[3, 4,7,8]], axis=1)
# having too much dataset, choose all years > 2010 to use
t4 = t4[t4['year']>2014] #198 rows and 6 columns
t4.head() 

,set_num,name_set,year,id,name_theme,quantity
467,71017-1,Lobster-Lovin’ Batman™,2017,484,Batman,1
468,71017-10,Pink Power Batgirl™,2017,484,Batman,1
469,71017-11,Red Hood™,2017,484,Batman,1
470,71017-12,The Eraser™,2017,484,Batman,1
471,71017-13,Nurse Harley Quinn™,2017,484,Batman,1


In [73]:
# then join parts and color and categories, and then join sets with them

#join parts with part_categories
t2 = pd.merge(parts,part_category, left_on = 'part_cat_id', right_on = 'id', suffixes = ('_part','_category'))
#join t2 with color (t2 join inventory and inventory join color)
t3 = pd.merge(t2, invenotry_parts, on = 'part_num')
# drop useless columns
t3 = t3.drop(t3.columns[[2, 3, 5]], axis=1)
t3.head()

,part_num,name_part,name_category,color_id,quantity,is_spare
0,0687b1,Set 0687 Activity Booklet 1,Non-LEGO,9999,1,f
1,10016414,Sticker Sheet #1 for 41055-1,Non-LEGO,9999,1,t
2,10019stk01,Sticker for Set 10019 - (43274/4170393),Non-LEGO,9999,1,t
3,10026stk01,Sticker for Set 10026 - (44942/4184185),Non-LEGO,9999,1,t
4,10029stk01,Sticker for Set 10029 - (4216816),Non-LEGO,9999,1,t


In [74]:
t5 = pd.merge(t4, invenotry_parts, left_on = 'id', right_on = 'inventory_id', how = 'inner',suffixes = ('_inven','_parts'))
t3 = t3.drop(t3.columns[[3]], axis=1)
#t5.head()

In [78]:
df = pd.merge(t3,t5, on = 'part_num')
df.head()

,part_num,name_part,name_category,quantity,is_spare_x,set_num,name_set,year,id,name_theme,quantity_inven,inventory_id,color_id,quantity_parts,is_spare_y
0,6361stk01,Sticker for Set 6361 - (190145),Non-LEGO,1,t,75899-1,LaFerrari,2015,601,Speed Champions,1,601,9999,1,t
1,6361stk01,Sticker for Set 6361 - (190145),Non-LEGO,1,t,75908-1,458 Italia GT2,2015,601,Speed Champions,1,601,9999,1,t
2,6361stk01,Sticker for Set 6361 - (190145),Non-LEGO,1,t,75909-1,McLaren P1,2015,601,Speed Champions,1,601,9999,1,t
3,6361stk01,Sticker for Set 6361 - (190145),Non-LEGO,1,t,75910-1,Porsche 918 Spyder,2015,601,Speed Champions,1,601,9999,1,t
4,6361stk01,Sticker for Set 6361 - (190145),Non-LEGO,1,t,75911-1,McLaren Mercedes Pit Stop,2015,601,Speed Champions,1,601,9999,1,t


In [79]:
# drop useless column 
df = df.drop(df.columns[[0, 3,5,8,11,14]], axis=1)
# get column name and # of rows of df
print(len(df.index))

16561358


In [80]:
# divide training set, validation set and testing set
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.4)  #60% training test and 40% testing 

In [46]:
# check how many rows in the training dataset
print(len(train.index))

9936814


In [47]:
# check how many rows in the testing dataset
print(len(test.index))

6624544


In [81]:
train.head()

,name_part,name_category,is_spare_x,name_set,year,name_theme,quantity_inven,color_id,quantity_parts
2206072,Brick Special 1 x 1 with Stud on 1 Side,Bricks Special,f,Dribbal,2015,Series 6,1,4,1
15383388,Slope 30° 1 x 1 x 2/3 (Cheese Slope),Bricks Sloped,f,Vacation Batman™,2017,Batman,1,71,3
15573737,Slope 30° 1 x 1 x 2/3 (Cheese Slope),Bricks Sloped,f,Rogue,2016,Series 16 Minifigures,1,71,1
4408415,Plate 1 x 2,Plates,f,Clumsy Guy,2016,Series 15 Minifigures,1,27,1
596261,Plate Special 1 x 1 with Clip Light [Thick Ring],Plates Special,f,Glam Metal Batman™,2017,Batman,1,15,2


In [82]:
list(train) 
#9 columns, want to predict theme_name 
# based on category_name, 
# quantity_x(inven quantity), is_spare_x, setname, color, quantity_y(inventory_parts quantity)

['name_part',
 'name_category',
 'is_spare_x',
 'name_set',
 'year',
 'name_theme',
 'quantity_inven',
 'color_id',
 'quantity_parts']

In [83]:
# explanotary variables: both categorical and numeric
# response variable: categorical 


In [86]:
# unique values in category_name
sorted(df['name_category'].unique())

['Bars, Ladders and Fences',
 'Baseplates',
 'Bricks',
 'Bricks Curved',
 'Bricks Round and Cones',
 'Bricks Sloped',
 'Bricks Special',
 'Flags, Signs, Plastics and Cloth',
 'Hinges, Arms and Turntables',
 'Minifig Accessories',
 'Minifigs',
 'Non-LEGO',
 'Panels',
 'Plants and Animals',
 'Plates',
 'Plates Angled',
 'Plates Round and Dishes',
 'Plates Special',
 'Rock',
 'String, Bands and Reels',
 'Supports, Girders and Cranes',
 'Technic Bricks',
 'Technic Pins',
 'Technic Special',
 'Tiles',
 'Tiles Printed',
 'Tiles Special',
 'Transportation - Land',
 'Tubes and Hoses',
 'Wheels and Tyres',
 'Windows and Doors',
 'Windscreens and Fuselage']

In [88]:
# unique setname
sorted(df['name_set'].unique())

[' Spectre',
 '458 Italia GT2',
 'Alien Trooper',
 'André Schürrle (9)',
 'Animal Control Officer',
 'Astronaut',
 'Babysitter',
 'Ballerina',
 'Banana Guy',
 'Banshee',
 'Barbara Gordon™',
 'Bart',
 'Bastian Schweinsteiger (7)',
 'Benedikt Höwedes (4)',
 'Berp',
 'Carpenter',
 'Catman™',
 'Christoph Kramer (20)',
 'Clan of the Cave Batman™',
 'Classic King',
 'Clumsy Guy',
 'Comic Book Guy',
 'Commissioner Gordon™',
 'Cute Little Devil Set',
 'Cyborg',
 'Desert Warrior',
 'Dick Grayson™',
 'Disco Diva',
 'Dog Show Winner',
 'Dr Hibbert',
 'Dribbal',
 'Drillex Diamond Job',
 'Edna Krabappel',
 'Egyptian Warrior',
 'Evil Wizard',
 'Fairy Batman™',
 'Farmer',
 'Faun',
 'Fencer',
 'Fly Monster',
 'Flying Warrior',
 'Forx',
 'Frightening Knight',
 'Galaxy Trooper',
 'Gargoyle',
 'Glam Metal Batman™',
 'Goblin',
 'Groundskeeper Willie',
 'Gurggle',
 'Hans Moleman',
 'Hiker',
 'Homer',
 'Hot Dog Man',
 'Ice Queen',
 'Janitor',
 'Jewel Thief',
 'Joachim Löw',
 'Jérôme Boateng (17)',
 'Kendo F

In [89]:
# unique theme
sorted(df['name_theme'].unique())

['Batman',
 'DFB Minifigures',
 'Pirates III',
 'Series 13 Minifigures',
 'Series 14 Minifigures',
 'Series 15 Minifigures',
 'Series 16 Minifigures',
 'Series 6',
 'Speed Champions',
 'The Simpsons',
 'Ultra Agents']

In [ ]:
# which model can be applied? 
##  may be Naive Bayes